In [9]:
pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 3.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 5.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.2/242.2 kB 5.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76

In [16]:
from model import NCF
from typing import Tuple, Dict, Any
from preprocessing_utils import *
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
from tqdm import tqdm
from torch.optim import Adam
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import random
import wandb
import torch.nn as nn
import torch.nn.functional as F
from dataprep import *
from eval_funcs import *
import random
import gradio as gr

device = torch.device("cuda" if torch.cuda.is_available() else "mps")

path_2_150D_model = "/Volumes/DeepLearner/Search & Recommendation System/Steam RecSys Weights/Steam_recsys_150D_32bz_epoch_6.pt"
path_2_200D_model = "/Volumes/DeepLearner/Search & Recommendation System/Steam RecSys Weights/steam_recsys_200D_32bz_epoch_15.pt"
path_2_1000D_model = "/Volumes/DeepLearner/Search & Recommendation System/Steam RecSys Weights/Steam_recsys_1000D_128bz_epoch_20.pt"

#user_item_path = '/notebooks/australian_users_items_clean.json'
user_item_path = '/Volumes/DeepLearner/Search & Recommendation System/Data/australian_users_items_clean_v2.json'

device

device(type='mps')

In [2]:
train_loader, val_loader, test_loader, user_mapping, item_mapping, user_item_df, val_df = prepare_simple_data_loaders(user_item_path)

Number of unique user_id: 70834
Number of unique item_id: 10978
Missing user_idx: 0
Missing item_idx: 0


In [3]:
# Check current mappings
print("Current user mapping size:", len(user_mapping))
print("Current item mapping size:", len(item_mapping))

# Load state dict to check its size
state_dict = torch.load(path_2_150D_model, map_location=device)
print("State dict user embedding size:", state_dict['user_gmf_embedding.weight'].shape[0])
print("State dict item embedding size:", state_dict['item_gmf_embedding.weight'].shape[0])


Current user mapping size: 70834
Current item mapping size: 10978
State dict user embedding size: 70834
State dict item embedding size: 10978


In [4]:
model = NCF(num_users=len(user_mapping), num_items=len(item_mapping), embed_dim=150)

model.load_state_dict(torch.load(path_2_150D_model, map_location=device))

<All keys matched successfully>

In [14]:
def get_top_games_for_display(model, user_mapping, item_mapping, user_item_df, k=10):
    # Select a random user for demonstration
    random_user_id = random.choice(list(user_mapping.keys()))

    # Get the top-k actual games
    top_k_actual_games = get_actual_top_k_games(user_mapping[random_user_id], user_item_df, user_mapping, k)

    # Get the top-k predicted games
    top_k_predicted_games = get_predicted_top_k_games(model, random_user_id, user_mapping, item_mapping, user_item_df, k)

    # Prepare the results
    results = {
        "user_id": random_user_id,
        "actual_games": top_k_actual_games,
        "predicted_games": top_k_predicted_games
    }

    return results


In [17]:
import gradio as gr

def gradio_wrapper():
    results = get_top_games_for_display(model, user_mapping, item_mapping, user_item_df)
    return f"User ID: {results['user_id']}", f"Actual Games: {results['actual_games']}", f"Predicted Games: {results['predicted_games']}"

iface = gr.Interface(fn=gradio_wrapper, 
                     inputs=[],  # No inputs in this case
                     outputs=["text", "text", "text"])

iface.launch()


Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/gradio/routes.py", line 516, in predict
    output = await route_utils.call_process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/gradio/route_utils.py", line 219, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/gradio/blocks.py", line 1437, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/gradio/blocks.py", line 1109, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^